In [1]:
# !pip install mlxtend==0.19.0

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris

iris = load_iris()
x = iris.data
y = iris.target
knn = KNeighborsClassifier(n_neighbors=4)

In [3]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

## Forward Selection 

In [4]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

sfs = SFS(knn,
          k_features=3,
          forward=True,
          floating=False,
          scoring='accuracy',
          cv=0,
          n_jobs=-1
            )

feature_names = ('sepal length','sepal width','petal length','petal width')
sfs =sfs.fit(x,y,custom_feature_names=feature_names)

In [5]:
sfs.get_metric_dict()

C:\Users\trilok\Anaconda3\lib\site-packages\numpy\core\_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\trilok\Anaconda3\lib\site-packages\numpy\core\_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{1: {'feature_idx': (3,),
  'cv_scores': array([0.96]),
  'avg_score': 0.96,
  'feature_names': ('petal width',),
  'ci_bound': nan,
  'std_dev': 0.0,
  'std_err': nan},
 2: {'feature_idx': (2, 3),
  'cv_scores': array([0.97333333]),
  'avg_score': 0.9733333333333334,
  'feature_names': ('petal length', 'petal width'),
  'ci_bound': nan,
  'std_dev': 0.0,
  'std_err': nan},
 3: {'feature_idx': (1, 2, 3),
  'cv_scores': array([0.97333333]),
  'avg_score': 0.9733333333333334,
  'feature_names': ('sepal width', 'petal length', 'petal width'),
  'ci_bound': nan,
  'std_dev': 0.0,
  'std_err': nan}}

In [6]:
import pandas as pd
pd.DataFrame.from_dict(sfs.get_metric_dict()).T

C:\Users\trilok\Anaconda3\lib\site-packages\numpy\core\_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\trilok\Anaconda3\lib\site-packages\numpy\core\_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
1,"(3,)",[0.96],0.96,"(petal width,)",NaN,0.0,NaN
2,"(2, 3)",[0.9733333333333334],0.973333,"(petal length, petal width)",NaN,0.0,NaN
3,"(1, 2, 3)",[0.9733333333333334],0.973333,"(sepal width, petal length, petal width)",NaN,0.0,NaN


## Backward Selection 

In [7]:
sfs = SFS(knn,
          k_features=2,
          forward=False,
          scoring='accuracy',
          floating=False, # it'll do bidirectional wayif its true in big dataset
          cv=2,
          n_jobs=1
         )
sfs =sfs.fit(x,y,custom_feature_names=feature_names)

In [8]:
pd.DataFrame().from_dict(sfs.get_metric_dict()).T

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
4,"(0, 1, 2, 3)","[0.96, 0.9466666666666667]",0.953333,"(sepal length, sepal width, petal length, peta...",0.028684,0.006667,0.006667
3,"(1, 2, 3)","[0.9733333333333334, 0.9466666666666667]",0.96,"(sepal width, petal length, petal width)",0.057369,0.013333,0.013333
2,"(2, 3)","[0.96, 0.96]",0.96,"(petal length, petal width)",0.0,0.0,0.0


# Recursive Feature Elimination

In [9]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [10]:
dataset=load_boston()
df=pd.DataFrame(dataset.data)
df.columns=dataset.feature_names
df.target=np.asarray(dataset.target)
df['price']=pd.Series(df.target)
df.head()

C:\Users\trilok\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset 

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [11]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [12]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=0)

In [13]:
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV

In [14]:
lin_reg = LinearRegression()

In [15]:
rfe = RFE(lin_reg)
myvalues = rfe.fit(X_train,y_train)
myvalues.support_
myvalues.ranking_

print("no. of features :  %s" %(myvalues.n_features_))
print("Selected features :  %s" %(myvalues.support_))
print("feature Ranking :  %s" %(myvalues.ranking_))

no. of features :  6
Selected features :  [False False False  True  True  True False  True False False  True False
  True]
feature Ranking :  [2 4 7 1 1 1 8 1 3 5 1 6 1]
